## CATBOOST

In [20]:
!pip install catboost

In [27]:
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from catboost import CatBoostClassifier
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

In [28]:
def get_summary(df):
    print(f"Dataset Shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = df.isnull().sum().values    
    summary['Uniques'] = df.nunique().values
    summary['First Value'] = df.loc[0].values
    summary['Second Value'] = df.loc[1].values
    return summary

In [29]:
file_loc = "data.csv"

In [30]:
df = pd.read_csv(file_loc)

In [31]:
df.head()

ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0           5849.0                0.0         0.0             360.0   
1           4583.0             1508.0       128.0             360.0   
2           3000.0                0.0        66.0             360.0   
3           2583.0             2358.0       120.0             360.0   
4           6000.0                0.0       141.0             360.0   

  Credit_History  Loan_Status  
0            Yes            1  
1            Yes            0  
2            Yes            1  
3            Yes            1  
4            Yes            1

In [32]:
print ("Total number of rows in dataset = {}".format(df.shape[0]))
print ("Total number of columns in dataset = {}".format(df.shape[1]))

Total number of rows in dataset = 614
Total number of columns in dataset = 6


In [33]:
result = get_summary(df)
result

Dataset Shape: (614, 6)


Name   dtypes  Missing  Uniques First Value Second Value
0    ApplicantIncome  float64        2      503      5849.0       4583.0
1  CoapplicantIncome  float64        2      287         0.0       1508.0
2         LoanAmount  float64        3      203         0.0        128.0
3   Loan_Amount_Term  float64        2       11       360.0        360.0
4     Credit_History   object        0        2         Yes          Yes
5        Loan_Status    int64        0        2           1            0

In [34]:
target_col = "Loan_Status"
X = df.loc[:, df.columns != target_col]
y = df.loc[:, target_col]

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.8, 
                                                    random_state=42)

In [36]:
X_train.head()

ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
397           3033.0             1459.0        95.0             360.0   
254          16250.0                0.0       192.0             360.0   
217           3727.0             1775.0       131.0             360.0   
4             6000.0                0.0       141.0             360.0   
256           6045.0                0.0       115.0             360.0   

    Credit_History  
397            Yes  
254             No  
217            Yes  
4              Yes  
256             No

In [37]:
features = list(X_train.columns)

In [38]:
cat_features = ["Credit_History"]

In [39]:
model_cb = CatBoostClassifier(task_type='GPU', iterations=100, 
                              random_state = 2022, 
                              eval_metric="F1")

In [40]:
model_cb.fit(X_train, y_train, cat_features= cat_features, plot=True, 
             eval_set=(X_test, y_test))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.220894
0:	learn: 0.8936170	test: 0.8115942	best: 0.8115942 (0)	total: 60.9ms	remaining: 6.03s
1:	learn: 0.8677249	test: 0.8312413	best: 0.8312413 (1)	total: 112ms	remaining: 5.47s
2:	learn: 0.8888889	test: 0.8375350	best: 0.8375350 (2)	total: 162ms	remaining: 5.23s
3:	learn: 0.8795812	test: 0.8354430	best: 0.8375350 (2)	total: 238ms	remaining: 5.71s
4:	learn: 0.8677249	test: 0.8366197	best: 0.8375350 (2)	total: 301ms	remaining: 5.71s
5:	learn: 0.8617021	test: 0.8382560	best: 0.8382560 (5)	total: 365ms	remaining: 5.72s
6:	learn: 0.8677249	test: 0.8375350	best: 0.8382560 (5)	total: 435ms	remaining: 5.78s
7:	learn: 0.8677249	test: 0.8387097	best: 0.8387097 (7)	total: 507ms	remaining: 5.83s
8:	learn: 0.8677249	test: 0.8387097	best: 0.8387097 (7)	total: 696ms	remaining: 7.04s
9:	learn: 0.8736842	test: 0.8398876	best: 0.8398876 (9)	total: 775ms	remaining: 6.97s
10:	learn: 0.8736842	test: 0.8398876	best: 0.8398876 (9)	total: 841ms	remaining: 6.8s
11:	learn: 0.8736842	te

95:	learn: 0.9772727	test: 0.8133140	best: 0.8398876 (9)	total: 7.16s	remaining: 298ms
96:	learn: 0.9772727	test: 0.8098694	best: 0.8398876 (9)	total: 7.24s	remaining: 224ms
97:	learn: 0.9772727	test: 0.8098694	best: 0.8398876 (9)	total: 7.3s	remaining: 149ms
98:	learn: 0.9772727	test: 0.8098694	best: 0.8398876 (9)	total: 7.35s	remaining: 74.3ms
99:	learn: 0.9772727	test: 0.8086957	best: 0.8398876 (9)	total: 7.42s	remaining: 0us
bestTest = 0.8398876404
bestIteration = 9
Shrink model to first 10 iterations.


In [41]:
y_pred = model_cb.predict(X_test)

In [42]:
f1_score(y_test, y_pred)

0.8398876404494382

In [43]:
accuracy_score(y_test, y_pred)

0.7682926829268293